In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict

In [2]:
pred_df = pd.read_pickle('pred_eval.pkl')
pred_df

,pseudo_gen_seq,code_gen_seq,pred_code_gen_seq
0,"[let, [CPY], [CPY], [CPY], [CPY], [CPY], [CPY]...","[int, [CPY], [CPY], [CPY], [CPY], [CPY], [CPY]...","[int, [CPY], [CPY], [CPY], [CPY], [CPY], [CPY]..."
1,"[read, [CPY]]","[cin, >>, [CPY], ;]","[cin, >>, [CPY], ;]"
2,"[read, [CPY]]","[cin, >>, [CPY], ;]","[cin, >>, [CPY], ;]"
3,"[read, [CPY]]","[cin, >>, [CPY], ;]","[cin, >>, [CPY], ;]"
4,"[read, [CPY]]","[cin, >>, [CPY], ;]","[cin, >>, [CPY], ;]"
...,...,...,...
19175,"[[CPY], array, [CPY]]","[[CPY], (, [CPY], ,, [CPY], +, m, ), ;]","[[CPY], (, [CPY], ,, [CPY], )]"
19176,"[[CPY], [CPY]]","[[CPY], (, [CPY], ,, [CPY], +, m, ), ;]","[[CPY], [CPY], ;]"
19177,"[[CPY], [CPY]]","[[CPY], (, [CPY], ,, [CPY], +, m, ), ;]","[[CPY], [CPY], ;]"
19178,"[[CPY], [CPY], [CPY], [CPY], to, [CPY], exclus...","[[CPY], (, int, [CPY], [CPY], [CPY], ;, [CPY],...","[[CPY], (, int, [CPY], [CPY], [CPY], ;, [CPY],..."


## Accuracy scores

#### Strict accuracy

Exact sentence matches

In [3]:
strict_bools = pred_df.apply(lambda row: np.array_equal(row['code_gen_seq'], row['pred_code_gen_seq']), axis=1)
strict_accuracy = strict_bools.sum() / len(pred_df)
strict_accuracy

0.6952033368091762

In [4]:
def compare_cols(row, col1, col2):
    """
    Count of equal words in two rows 
    """
    try:
        comps = np.array(row['code_gen_seq']) == np.array(row['pred_code_gen_seq'])
    except ValueError as e:
        comps = np.array([False])
    return np.sum(comps)

In [5]:
word_matches = np.sum(pred_df.apply(compare_cols, args=('code_gen_seq', 'pred_code_gen_seq'), axis=1))
total_words = np.sum(pred_df.apply(lambda row: np.size(row['code_gen_seq']), axis=1))

word_accuracy = word_matches / total_words
word_accuracy

0.5746113225359176

## Combine

In [7]:
eval_df = pd.read_csv('../../data/input-tok-eval.tsv', sep='\t', header=None)
eval_df.columns = ['pseudo', 'code']
eval_df

,pseudo,code
0,"let a , b , c , d , e be integers","int a , b , c , d , e ;"
1,read a,cin >> a ;
2,read b,cin >> b ;
3,read c,cin >> c ;
4,read d,cin >> d ;
...,...,...
19175,sort array a,"sort ( a , a + m ) ;"
19176,sort b,"sort ( b , b + m ) ;"
19177,reverse b,"reverse ( b , b + m ) ;"
19178,"for i = 0 to m exclusive , set cnt [ b [ i ] ....",for ( int i = 0 ; i < m ; ++ i ) { cnt [ b [ i...


In [8]:
cpy_dataset = pd.read_pickle('../../data/CPY_dataset_eval.pkl')
cpy_dataset = cpy_dataset.drop(columns=['code_gen_seq', 'code_gen_seq_aug'])
cpy_dataset

,pseudo_gen_seq
0,"[let, [CPY], [CPY], [CPY], [CPY], [CPY], [CPY]..."
1,"[read, [CPY]]"
2,"[read, [CPY]]"
3,"[read, [CPY]]"
4,"[read, [CPY]]"
...,...
19175,"[[CPY], array, [CPY]]"
19176,"[[CPY], [CPY]]"
19177,"[[CPY], [CPY]]"
19178,"[[CPY], [CPY], [CPY], [CPY], to, [CPY], exclus..."


In [9]:
import importlib
import preprocess
importlib.reload(preprocess)

<module 'preprocess' from '/Users/vibhamasti/Personal/College/Sem_6/NLP/Pseudocode_to_code/spoc/copy_generation/combine/preprocess.py'>

In [10]:
preprocess.tokenize_column(eval_df, col_to_tokenize='code', tokenized_col_name='code_tokens', inplace=True)
preprocess.tokenize_column(eval_df, col_to_tokenize='pseudo', tokenized_col_name='pseudo_tokens', inplace=True)
eval_df.drop(columns=['code', 'pseudo'], inplace=True)
eval_df

,code_tokens,pseudo_tokens
0,"[int, a, ,, b, ,, c, ,, d, ,, e, ;]","[let, a, ,, b, ,, c, ,, d, ,, e, be, integers]"
1,"[cin, >>, a, ;]","[read, a]"
2,"[cin, >>, b, ;]","[read, b]"
3,"[cin, >>, c, ;]","[read, c]"
4,"[cin, >>, d, ;]","[read, d]"
...,...,...
19175,"[sort, (, a, ,, a, +, m, ), ;]","[sort, array, a]"
19176,"[sort, (, b, ,, b, +, m, ), ;]","[sort, b]"
19177,"[reverse, (, b, ,, b, +, m, ), ;]","[reverse, b]"
19178,"[for, (, int, i, =, 0, ;, i, <, m, ;, ++, i, )...","[for, i, =, 0, to, m, exclusive, ,, set, cnt, ..."


In [11]:
X_eval = pd.concat([eval_df[['pseudo_tokens']], pred_df[['pred_code_gen_seq']], cpy_dataset[['pseudo_gen_seq']]], axis=1)
y_eval = eval_df[['code_tokens']]

In [12]:
X_eval.head()

,pseudo_tokens,pred_code_gen_seq,pseudo_gen_seq
0,"[let, a, ,, b, ,, c, ,, d, ,, e, be, integers]","[int, [CPY], [CPY], [CPY], [CPY], [CPY], [CPY]...","[let, [CPY], [CPY], [CPY], [CPY], [CPY], [CPY]..."
1,"[read, a]","[cin, >>, [CPY], ;]","[read, [CPY]]"
2,"[read, b]","[cin, >>, [CPY], ;]","[read, [CPY]]"
3,"[read, c]","[cin, >>, [CPY], ;]","[read, [CPY]]"
4,"[read, d]","[cin, >>, [CPY], ;]","[read, [CPY]]"


In [13]:
y_eval.head()

,code_tokens
0,"[int, a, ,, b, ,, c, ,, d, ,, e, ;]"
1,"[cin, >>, a, ;]"
2,"[cin, >>, b, ;]"
3,"[cin, >>, c, ;]"
4,"[cin, >>, d, ;]"


In [14]:
input_words = pickle.load(open('input_words.pkl', 'rb'))
output_words = pickle.load(open('output_words.pkl', 'rb'))

pseudo_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
code_token_index = dict([(word, i+1) for i, word in enumerate(output_words)])

reverse_pseudo_index = dict((i, word) for word, i in pseudo_token_index.items())
reverse_code_index = dict((i, word) for word, i in code_token_index.items())

In [15]:
# Decision tree?
# print(pseudo_token_index['[CPY]'])
# print(pseudo_token_index['[UNK]'])

In [17]:
input_lexicon = defaultdict(lambda: pseudo_token_index[''], pseudo_token_index)
output_lexicon = defaultdict(lambda: code_token_index[''], code_token_index)

In [18]:
# input_lexicon['==']

In [19]:
importlib.reload(preprocess)
X_eval['features'] = X_eval.apply(lambda row: preprocess.create_combine_features(row, 'pred_code_gen_seq', 'pseudo_tokens', input_lexicon), axis=1)
X_eval.head()

,pseudo_tokens,pred_code_gen_seq,pseudo_gen_seq,features
0,"[let, a, ,, b, ,, c, ,, d, ,, e, be, integers]","[int, [CPY], [CPY], [CPY], [CPY], [CPY], [CPY]...","[let, [CPY], [CPY], [CPY], [CPY], [CPY], [CPY]...","[{'word': 1057, 'length': 3, 'is_numeric': Fal..."
1,"[read, a]","[cin, >>, [CPY], ;]","[read, [CPY]]","[{'word': 1, 'length': 3, 'is_numeric': False,..."
2,"[read, b]","[cin, >>, [CPY], ;]","[read, [CPY]]","[{'word': 1, 'length': 3, 'is_numeric': False,..."
3,"[read, c]","[cin, >>, [CPY], ;]","[read, [CPY]]","[{'word': 1, 'length': 3, 'is_numeric': False,..."
4,"[read, d]","[cin, >>, [CPY], ;]","[read, [CPY]]","[{'word': 1, 'length': 3, 'is_numeric': False,..."


In [20]:
def copy_tags(row: pd.Series):
    """
    Combine the copy tags for a row
    """
    combination = []
    tokens_to_copy = []

    for i, token in enumerate(row['pseudo_gen_seq']):
        if token == '[CPY]':
            tokens_to_copy.append(row['pseudo_tokens'][i])

    token_iter =  iter(tokens_to_copy)

    for i, token in enumerate(row['pred_code_gen_seq']):
        if token == '[CPY]':
            try:
                combination.append(next(token_iter))
            except StopIteration:
                combination.append('')
        else:
            combination.append(token)
    
    return combination

In [21]:
y_eval = pd.concat([eval_df[['code_tokens']], X_eval.apply(copy_tags, axis=1)], axis=1)
y_eval.columns = ['code_tokens', 'code_gen_seq']

In [22]:
y_eval

,code_tokens,code_gen_seq
0,"[int, a, ,, b, ,, c, ,, d, ,, e, ;]","[int, a, ,, b, ,, c, ,, d, ,, e, ;]"
1,"[cin, >>, a, ;]","[cin, >>, a, ;]"
2,"[cin, >>, b, ;]","[cin, >>, b, ;]"
3,"[cin, >>, c, ;]","[cin, >>, c, ;]"
4,"[cin, >>, d, ;]","[cin, >>, d, ;]"
...,...,...
19175,"[sort, (, a, ,, a, +, m, ), ;]","[sort, (, a, ,, , )]"
19176,"[sort, (, b, ,, b, +, m, ), ;]","[sort, b, ;]"
19177,"[reverse, (, b, ,, b, +, m, ), ;]","[reverse, b, ;]"
19178,"[for, (, int, i, =, 0, ;, i, <, m, ;, ++, i, )...","[for, (, int, i, =, 0, ;, m, <, cnt, ;, [, ++,..."


## Strict accuracy

In [23]:
strict_bools = y_eval.apply(lambda row: np.array_equal(row['code_tokens'], row['code_gen_seq']), axis=1)
strict_accuracy = strict_bools.sum() / len(y_eval)

In [24]:
strict_accuracy

0.5697601668404588

In [131]:
# importlib.reload(preprocess)
# y_eval['code_indices'] = y_eval.apply(lambda row: preprocess.create_indices(row, 'code_tokens', output_lexicon), axis=1)
# y_eval.head()

In [132]:

# from sklearn.tree import DecisionTreeClassifier
# from sklearn.feature_extraction import DictVectorizer
# from sklearn.pipeline import Pipeline

In [133]:
# X_eval_total = [word for l in X_eval['features'].tolist() for word in l]
# y_eval_total = [word for l in y_eval['code_indices'].tolist() for word in l]

In [134]:
# clf = Pipeline([
#     ('vectorizer', DictVectorizer()),
#     ('classifier', DecisionTreeClassifier(criterion='entropy'))
# ])

# clf.fit(X_eval_total, y_eval_total)   # Use only the first 10K samples if you're running it multiple times. It takes a fair bit :)
 
# print('evaling completed')

### Seq2Seq

In [ ]:
# from keras.models import 

In [ ]:
# def seq2seq_model_builder():
#     encoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
#     encoder_embedding = embed_layer(encoder_inputs)
#     encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
#     encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
    
#     decoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
#     decoder_embedding = embed_layer(decoder_inputs)
#     decoder_LSTM = LSTM(HIDDEN_DIM, return_state=True, return_sequences=True)
#     decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])
    
#     # dense_layer = Dense(VOCAB_SIZE, activation='softmax')
#     outputs = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))(decoder_outputs)
#     model = Model([encoder_inputs, decoder_inputs], outputs)
    
#     return model